In [ ]:
# Project: Energy Consumption Prediction
# Phase 1: Project Setup and Initial Data Exploration

# 1. Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set plot style for better aesthetics
# 'fivethirtyeight' is a popular choice for data visualisation
plt.style.use('fivethirtyeight')

# 2. Load the Dataset
try:
    df = pd.read_csv('data/PJME_hourly.csv')
    # df = pd.read_csv('sample_data/PJME_hourly.csv') # for Colab
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: PJME_hourly.csv not found. Make sure it is in the 'data/' directory.")
    print("Current working directory:", os.getcwd()) # for debugging if there is a path issue

# Display the first few rows
print("\nFirst five rows of the dataset:")
print(df.head())

# Get the earliest and latest date and time in Datetime column
print("\nEarliest date and time:", min(df['Datetime']))
print("Latest date and time:", max(df['Datetime']))

# Get a summary of the DataFrame
print("\nDataFrame Information:")
print(df.info())

# Get descriptive statistics for numerical columns
print("\nDescriptive Statistics:")
print(df.describe())

# 3. Handling Missing Values (if any)
# Check for missing values
print("\nMissing values before processing:")
print(df.isnull().sum())

In [ ]:
# 4. Convert 'Datetime' Column to Datetime Object and Set as Index
# Convert 'Datetime' column to datetime objects
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Set 'Datetime' as the DataFrame index and rename it
df = df.set_index('Datetime').rename_axis('Datetime')

# Ensure the index is sorted chronologically
df = df.sort_index()

# Check the new information to confirm datetime index
print("\nDataFrame Information after Datetime conversion and index setting:")
print(df.info())

# Check for any duplicate timestamps
# Should ideally be none for timestamps
print("\nNumber of duplicate timestamps:", df.index.duplicated().sum())

# Check for frequency consistency
# If the data is truly hourly, the difference between consecutive timestamps should be 1 hour
df.index.to_series().diff().value_counts()

In [ ]:
# Handle the 4 duplicate timestamps and the missing hours

print("Original shape:", df.shape)

# Handle duplicate timestamps
# Keep the first occurrence for duplicates
df = df.loc[~df.index.duplicated(keep='first')]
print("Shape after removing duplicate timestamps:", df.shape)

# Create a complete, continuous hourly DateTimeIndex for the entire period
# This fills in any missing hours such as those 2-hour gaps due to Daylight Saving Time transitions
# Find the min and max datetime in the data
start_date = df.index.min()
end_date = df.index.max()

# Generate a complete hourly date range
full_date_range = pd.date_range(start=start_date, end=end_date, freq='h')

# Reindex the DataFrame to this full date range
# This inserts NaN for any missing hours
df = df.reindex(full_date_range)
print("Shape after reindexing to full hourly range:", df.shape)

# Handle missing values introduced by reindexing (due to two-hour gaps)
# Fill in any actual missing hours that are NaN
# For energy consumption, interpolating is often a good choice (especially linear or spline)
print("\nMissing values after reindexing (these are previously 'skipped' hours):")
print(df.isnull().sum())

# Interpolate the missing values
df['PJME_MW'] = df['PJME_MW'].interpolate(method='linear', limit_direction='both', limit_area='inside')

print("\nMissing values after interpolation:")
print(df.isnull().sum())

# Verify the index frequency again (should be perfectly hourly now)
print("\nValue counts of item differences after full processing:")
print(df.index.to_series().diff().value_counts())

In [ ]:
# 5. Initial Visualisations using Exploratory Data Analysis (EDA)

# Create the 'images' directory if it does not exist yet
if not os.path.exists('images'):
    os.makedirs('images')

# Plot the entire time series
fig, ax = plt.subplots(figsize=(15, 6))
df['PJME_MW'].plot(ax=ax, title='PJME Hourly Energy Consumption (MW)')
ax.set_xlabel('Date')
ax.set_ylabel('Consumption (MW)')
plt.grid(True)
plt.tight_layout()
plt.savefig('images/ph1_entire_time_series.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

# Zoom in on a specific period to see daily/weekly patterns
# Example: May 2015
fig, ax = plt.subplots(figsize=(15, 6))
df.loc['2015-03'].plot(ax=ax, title='PJME Hourly Energy Consumption in March 2015')
ax.set_xlabel('Date')
ax.set_ylabel('Consumption (MW)')
plt.grid(True)
plt.tight_layout()
plt.savefig('images/ph1_zoom_period_time_series.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

# Further zoom in: A single week in 2015
fig, ax = plt.subplots(figsize=(15, 6))
df.loc['2015-03-01':'2015-03-07'].plot(ax=ax, title='PJME Hourly Energy Consumption in the First Week of March 2015')
ax.set_xlabel('Date')
ax.set_ylabel('Consumption (MW)')
plt.grid(True)
plt.tight_layout()
plt.savefig('images/ph1_zoom_week_time_series.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

In [ ]:
# Phase 2: Feature Engineering

# 1. Import Additional Library
# Install the 'holidays' library using CLI if it is not done already
# pip install holidays
import holidays

print("Additional feature engineering library imported successfully.")

In [ ]:
# 2.1 Time-Based Features
# These are features that are derived from the existing DatetimeIndex that help the model understand
# hourly, daily, weekly, and annual seasonality.

# Create features based on the datetime index
df['hour_of_day'] = df.index.hour
df['day_of_week'] = df.index.dayofweek # 0 for Monday, 6 for Sunday
df['day_of_year'] = df.index.dayofyear
df['week_of_year'] = df.index.isocalendar().week.astype(int) # using isocalendar for week number
df['month'] = df.index.month
df['quarter'] = df.index.quarter
df['year'] = df.index.year
df['is_weekend'] = (df.index.dayofweek >= 5).astype(int) # 1 for weekend (Saturday, Sunday), 0 for weekday
df['day_of_month'] = df.index.day # the day of the month (1-31)

# Create cyclical features for hour, day of year, etc
# These help the models that do not inherently understand cycles (such as linear models)
df['hour_sin'] = np.sin(2 * np.pi * df['hour_of_day'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour_of_day'] / 24)
df['dayofyear_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
df['dayofyear_cos'] = np.cos(2* np.pi * df['day_of_year'] / 365)

# Re-assert the index column name
df.rename_axis('Datetime', inplace=True)

print("DataFrame after adding time-based features (first 5 rows):")
print(df.head())
print("\nUnique values for a few of the new features:")
print(df['hour_of_day'].unique())
print(df['day_of_week'].unique())
print(df['is_weekend'].value_counts())

In [ ]:
# 2.2 Lagged Features
# These features provide information about past energy consumption, which is highly indicative of future consumption.

# Create lagged features
# Consumption from the previous hour
df['lag_1_hour'] = df['PJME_MW'].shift(1)

# Consumption from the same hour on the previous day (24 hours before)
df['lag_24_hour'] = df['PJME_MW'].shift(24)

# Consumption from the same hour on the previous week (168 hours before)
df['lag_168_hour'] = df['PJME_MW'].shift(168)

# There are NaN values at the beginning of these shifted features, which are expected since there is no other data
# to shift from the past. They will need to be handled before modelling by dropping or imputing them carefully.
print("\nDataFrame after adding lagged features (first few rows - will have NaNs):")
print(df.head(200))
print("\nNumber of NaNs in lagged features:")
print(df[['lag_1_hour', 'lag_24_hour', 'lag_168_hour']].isnull().sum())

In [ ]:
# 2.3 Public Holidays
# Define the country and states for PJM (mostly US, specific states)
# PJM covers all or parts of DE, IL, IN, KY, MD, MI, NC, NJ, OH, PA, TN, VA, WV, and Washington D.C.
# Start with US federal holidays for simplicity.

# Adjust the years to match the dataset's range of 2002 to 2018
us_holidays = holidays.US(years=range(2002, 2019))

# Create an 'is_holiday' column based on the DataFrame's index dates
# The index only contains the date part for matching with the holidays library
df['date_only'] = df.index.date
df['is_holiday'] = df['date_only'].apply(lambda x: 1 if x in us_holidays else 0)

# Drop the temporary 'date_only' column
df = df.drop(columns=['date_only'])

print("\nDataFrame after adding 'is_holiday' feature (showing a few holiday entries):")
print(df[df['is_holiday'] == 1].head())
print("\nNumber of holidays found:", df['is_holiday'].sum())

In [ ]:
# 2.4 Weather Data (Temperature)
# Use historical hourly temperature data for a representative location within the PJM service territory.
# Data is retrieved from Open-Meteo by using their API
# Selected region: Chicago
# Latitude: 41.8832, longitude: -87.6324

# Retrieve historical temperature data from Open-Meteo

import requests

# Define the coordinates for Chicago
latitude = 41.8832
longitude = -87.6324

# Define the date range
start_date = '2002-01-01'
end_date = '2018-08-03'

# Open-Meteo Historical Weather API endpoint
url = "https://archive-api.open-meteo.com/v1/archive"

params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": start_date,
    "end_date": end_date,
    "hourly": "temperature_2m", # request hourly temperature at 2 meters
    "timezone": "America/Chicago"
}

response = requests.get(url, params=params)
data = response.json()

# Ensure the energy consumption data has an index name
df.index.name = 'Datetime'

# Check if the API call was successful
if 'hourly' in data:
    temp_df = pd.DataFrame(data['hourly'])
    temp_df['time'] = pd.to_datetime(temp_df['time'])
    temp_df = temp_df.set_index('time')
    temp_df.index.name = 'Datetime'
    temp_df = temp_df.rename(columns={'temperature_2m': 'current_temp'}) # rename column for temperature at 2 meters
    print("Temperature data loaded successfully.")
    print(temp_df.head())
    print(temp_df.info())
else:
    print("Error fetching temperature data:", data)

In [ ]:
# Merge temp_df with the main energy consumption DataFrame (df)
df = df.merge(temp_df[['current_temp']], left_index=True, right_index=True, how='left')
df.rename(columns={'current_temp': 'temperature'}, inplace=True)

print("Merged df (first 5 rows):")
print(df.head())

In [ ]:
# Create temperature-derived features

# Lagged temperature feature
df['lag_24_temp'] = df['temperature'].shift(24)

# 3-day rolling average temperature
df['rolling_72_temp_avg'] = df['temperature'].rolling(window=72).mean() # 3-day rolling average

# Squared temperature terms
df['temp_squared'] = np.power(df['temperature'], 2)

# There are NaN values at the beginning of these shifted features, which are expected since there is no other data
# to shift from the past. They will need to be handled before modelling by dropping or imputing them carefully.
print("\nDataFrame after adding temperature-derived features (first few rows - will have NaNs):")
print(df.head(200))
print("\nNumber of NaNs in temperature_derived features:")
print(df[['lag_24_temp', 'rolling_72_temp_avg', 'temp_squared']].isnull().sum())

In [ ]:
# Drop the NaN values
# Given that the dataset covers years of hourly data from 2002 to 2018, which is over 140,000 hours, the maximum number
# of NaN values is very small in relative to the entire dataset size (0.12%), so it is acceptable to drop the rows with
# NaN values for time series data.

# Create a list of all feature columns that may contain NaN values
feature_columns_to_check = [
    'lag_1_hour', 'lag_24_hour', 'lag_168_hour', 'lag_24_temp', 'rolling_72_temp_avg', 'temp_squared'
]

# Drop rows where any of these specified feature columns have NaN values
df.dropna(subset=feature_columns_to_check, inplace=True)

# Verify that the rows with NaN values are gone
print("Number of NaNs after dropping:")
print(df[feature_columns_to_check].isnull().sum())

In [ ]:
print("Columns in df BEFORE data splitting:")
print(df.columns)

In [ ]:
# Visualise selective key features

# Set a consistent plot style for better aesthetics
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 8)

# PJME_MW over time
# Overall trends of energy consumption, focusing on a recent 2-year period for clarity
# As plotting the entire dataset could be overwhelming, a subset is used the initial exploration.
df['PJME_MW'].loc['2016':'2017'].plot(
    title='PJME_MW Consumption in 2016 and 2017',
    ylabel='Energy (MW)',
    color='chocolate'
)
plt.savefig('images/ph2_pjme_mw_over_time.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

# PJME_MW vs Hour of Day (by Weekday/Weekend)
# The impact of energy consumption based on hours of the day and differentiated by weekdays and weekends
df['is_weekend_label'] = df['is_weekend'].map({0: 'No', 1: 'Yes'})

fig, ax = plt.subplots(figsize=(15, 6))

sns.boxplot(
    data=df,
    x='hour_of_day',
    y='PJME_MW',
    hue='is_weekend_label', # differentiate between weekdays (0) and weekends (1)
    ax=ax,
    palette='viridis'
)

ax.set_title('PJME_MW Consumption by Hour of Day (Weekday vs Weekend)')
ax.set_xlabel('Hour of Day')
ax.set_ylabel('Energy (MW)')
plt.legend(title='Is Weekend?')
plt.savefig('images/ph2_pjme_mw_vs_hour_of_day.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

# PJME_MW vs Temperature
# Direct correlation between energy demand and temperature of day
fig, ax = plt.subplots(figsize=(15, 6))

sns.scatterplot(
    data=df,
    x='temperature',
    y='PJME_MW',
    alpha=0.2, # handle overplotting for large datasets
    ax=ax,
    color='teal'
)

ax.set_title('PJME_MW vs Temperature')
ax.set_xlabel('Temperature (°C)')
ax.set_ylabel('Energy (MW)')
plt.savefig('images/ph2_pjme_mw_vs_temperature.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

# Average PJME_MW by Hour and Day of Week
# Typical demand for every hour of every day of the week
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14

pivot_table = df.pivot_table(values='PJME_MW', index='hour_of_day', columns='day_of_week')
day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

fig, ax = plt.subplots(figsize=(12, 8))

sns.heatmap(pivot_table,
            cmap='magma',
            annot=True, # display the annotation numbers
            fmt=".0f", # format the numbers as integers
            linewidths=0.5,
            annot_kws={"fontsize": 10},
            ax=ax,
            cbar_kws={'label': 'Energy (MW)'}
           )

plt.title('Average PJME_MW by Hour of Day and Day of Week', fontsize=plt.rcParams['axes.titlesize'])
plt.xlabel('Day of Week', fontsize=plt.rcParams['axes.labelsize'])
plt.ylabel('Hour of Day', fontsize=plt.rcParams['axes.labelsize'])

ax.set_xticklabels(day_names)
ax.tick_params(axis='x', labelsize=10)
ax.tick_params(axis='y', labelsize=10)
cbar_ax = fig.axes[-1]
cbar_ax.tick_params(labelsize=8)
plt.savefig('images/ph2_average_pjme_mw_by_hour_and_day_of_week.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

In [ ]:
# Phase 3: Model Selection and Training

# 1. Import Additional Libraries
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lightgbm.callback import early_stopping

print("\nAdditional model selection and training libraries imported successfully.")

In [ ]:
# 2. Data Splitting and Feature/Target Definition
# A chronological split with a cutoff date is the approach adopted to prepare the dataset for model training and
# evaluation. This method is critical for time series data to prevent data leakage, which would otherwise occur with
# random splitting. Data leakage exposes the model to "future information" during training, leading to overly optimistic
# performance estimates on the test set.
# For this project, our choice of cutoff date for the test set is 01 January 2017, which roughly encompasses the last
# 1.5 years of the dataset.

# Define the cutoff date
cutoff_date = '2017-01-01'

# Define the features (X) and target (y)
# The target variable is 'PJME_MW'
target = 'PJME_MW'

# List out all final features excluding the target variable
# Ensure that only the intended features are used and makes the model's inputs very clear for everyone
# Check the list of feature columns: features = [col for col in df.columns if col != target]

features = [
    'hour_of_day', 'day_of_week', 'day_of_year', 'week_of_year', 'month', 'quarter', 'year', 'is_weekend',
    'day_of_month', 'hour_sin', 'hour_cos', 'dayofyear_sin', 'dayofyear_cos', 'lag_1_hour', 'lag_24_hour', 'lag_168_hour',
    'is_holiday', 'temperature', 'lag_24_temp', 'rolling_72_temp_avg', 'temp_squared']

# Split the data chronologically using the explicit feature list
X_train = df[df.index < cutoff_date][features]
y_train = df[df.index < cutoff_date][target]

X_test = df[df.index >= cutoff_date][features]
y_test = df[df.index >= cutoff_date][target]

print(f"Full data shape: {df.shape}")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

print(f"\nTrain data range: {X_train.index.min()} to {X_train.index.max()}")
print(f"Test data range: {X_test.index.min()} to {X_test.index.max()}")

# Verify that there is no overlap
if X_train.index.max() < X_test.index.min():
    print("\nData split verified: No temporal overlap between train and test sets.")
else:
    print("\nWarning: Temporal overlap detected. Review your cutoff date or splitting logic.")

In [ ]:
print("\nColumns in X_test AFTER data splitting:")
print(X_test.columns)

In [ ]:
# 3. Initial Model Training (for Baseline and Feature Importance)
# Powerful gradient boosting models such as XGBoost and LightGBM are excellent model choices to start with for time
# series forecasting as they can handle complex relationships well and provide feature importance scores.

# --- XGBoost Model ---

# Initialise the model
# For initial training, we can use default or slightly adjusted parameters.
# 'n_estimators' is the number of boosting rounds (trees)
# 'early_stopping_rounds' can prevent overfitting during training
baseline_xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror', # objective for regression tasks
    n_estimators=1000, # number of boosting rounds (trees)
    learning_rate=0.05, # step size shrinkage to prevent overfitting
    random_state=42, # for reproducibility
    n_jobs=-1, # to use all available CPU cores
    eval_metric='rmse', # evaluate using Root Mean Squared Error
    early_stopping_rounds=50
)

# Train the model
# Use early_stopping_rounds to prevent overfitting and eval_set to monitor performance on a validation set
# We will use X_test as a simple validation here but for true tuning, it would be more ideal to use a separate validation
# split from the training data or a time series cross-validation strategy.
# For baseline, using X_test as eval_set is acceptable to see initial performance.
baseline_xgb_model.fit(X_train, y_train,
                       eval_set=[(X_train, y_train), (X_test, y_test)],
                       verbose=False # surpress all printing during training
             )

# Make predictions on the test set
baseline_xgb_y_pred = baseline_xgb_model.predict(X_test)

# Evaluate the baseline model's performance
baseline_xgb_rmse = np.sqrt(mean_squared_error(y_test, baseline_xgb_y_pred))
baseline_xgb_mae = mean_absolute_error(y_test, baseline_xgb_y_pred)
baseline_xgb_r2 = r2_score(y_test, baseline_xgb_y_pred)

print(f"XGBoost Baseline Model Performance:")
print(f"Root Mean Squared Error (RMSE): {baseline_xgb_rmse:.2f}")
print(f"Mean Absolute Error (MAE): {baseline_xgb_mae:.2f}")
print(f"R-squared (R^2): {baseline_xgb_r2:.5f}")

# Visualise Predictions vs Actuals
# Create a DataFrame for easy plotting
baseline_xgb_results = pd.DataFrame({'Actual': y_test, 'Predicted': baseline_xgb_y_pred}, index=y_test.index)

plt.figure(figsize=(15, 6))
plt.plot(baseline_xgb_results.index, baseline_xgb_results['Actual'], label='Actual Consumption', alpha=0.7)
plt.plot(baseline_xgb_results.index, baseline_xgb_results['Predicted'], label='Predicted Consumption', alpha=0.8)
plt.title('XGBoost Baseline Model: Actual vs Predicted Energy Consumption')
plt.xlabel('Datetime')
plt.ylabel('PJME_MW')
plt.legend()
plt.grid(True)
plt.savefig('images/ph3_xgboost_baseline_model.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

In [ ]:
# Extract feature importances
baseline_xgb_feature_importances = pd.DataFrame(
    {'feature': X_train.columns, 'importance': baseline_xgb_model.feature_importances_}
).sort_values(by='importance', ascending=False)

print("\nTop 10 Feature Importances from Baseline XGBoost Model:")
print(baseline_xgb_feature_importances.head(10))

# Visualise all feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=baseline_xgb_feature_importances.head(20)) # top 20 features
plt.title('XGBoost Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.savefig('images/ph3_top_10_features_xgboost_baseline_model.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

In [ ]:
# --- LightGBM Model ---

# Initialise the model
# 'n_estimators' is the number of boosting rounds (trees)
baseline_lgbm_model = lgb.LGBMRegressor(
    objective='regression', # objective for regression tasks
    n_estimators=1000, # number of boosting rounds (trees)
    learning_rate=0.05, # step size shrinkage to prevent overfitting
    random_state=42, # for reproducibility
    n_jobs=-1 # to use all available CPU cores
)

# Define the early stopping callback
early_stopping_callback_lgbm = lgb.early_stopping(
    stopping_rounds=50, # number of rounds with no improvement to stop
    verbose=False # suppress printing during training
)

# Train the model
# Use eval_set and callbacks to monitor performance on a validation set
baseline_lgbm_model.fit(X_train, y_train,
              eval_set=[(X_test, y_test)], # using X_test for validation
              eval_metric='rmse' # evaluate using Root Mean Squared Error
              )

# Make predictions on the test set
baseline_lgbm_y_pred = baseline_lgbm_model.predict(X_test)

# Evaluate the baseline model's performance
baseline_lgbm_rmse = np.sqrt(mean_squared_error(y_test, baseline_lgbm_y_pred))
baseline_lgbm_mae = mean_absolute_error(y_test, baseline_lgbm_y_pred)
baseline_lgbm_r2 = r2_score(y_test, baseline_lgbm_y_pred)

print(f"\nLightGBM Baseline Model Performance:")
print(f"Root Mean Squared Error (RMSE): {baseline_lgbm_rmse:.2f}")
print(f"Mean Absolute Error (MAE): {baseline_lgbm_mae:.2f}")
print(f"R-squared (R^2): {baseline_lgbm_r2:.5f}")

# Visualise Predictions vs Actuals
# Create a DataFrame for easy plotting
baseline_lgbm_results = pd.DataFrame({'Actual': y_test, 'Predicted': baseline_lgbm_y_pred}, index=y_test.index)

plt.figure(figsize=(15, 6))
plt.plot(baseline_lgbm_results.index, baseline_lgbm_results['Actual'], label='Actual Consumption', alpha=0.7)
plt.plot(baseline_lgbm_results.index, baseline_lgbm_results['Predicted'], label='Predicted Consumption', alpha=0.8)
plt.title('LightGBM Baseline Model: Actual vs Predicted Energy Consumption')
plt.xlabel('Datetime')
plt.ylabel('PJME_MW')
plt.legend()
plt.grid(True)
plt.savefig('images/ph3_lightgbm_baseline_model.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

In [ ]:
# Extract feature importances
baseline_lgbm_feature_importances = pd.DataFrame(
    {'feature': X_train.columns, 'importance': baseline_lgbm_model.feature_importances_}
).sort_values(by='importance', ascending=False)

print("\nTop 10 Feature Importances from Baseline LightGBM Model:")
print(baseline_lgbm_feature_importances.head(10))

# Visualise all feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=baseline_lgbm_feature_importances.head(20)) # top 20 features
plt.title('LightGBM Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.savefig('images/ph3_top_10_features_lightgbm_baseline_model.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

In [ ]:
# Phase 4: Advanced Feature Engineering

# 1.1 Derived Weather Features (Heating and Cooling Degree Days)
# Capture more complex, non-linear and domain-specific relationships between weather conditions and
# energy consumption as compared to using raw temperature data alone.
# The base temperature used is 18°C.

# Check if 'Datetime' column exists. Set it if it is not the index already
if 'Datetime' in df.columns:
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df = df.set_index('Datetime')
# Convert it if 'Datetime' is already the index but its dtype isn't datetime64[ns]
elif not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index)

# Define a base temperature
# 18 degrees Celsius is common for energy forecasting
base_temperature_celsius = 18

# Calculate daily average temperature
# Resample will work correctly now that we're sure df.index is a DatetimeIndex
daily_temp_avg = df['temperature'].resample('D').mean()

# Calculate HDD and CDD
df_daily_hdd_cdd = pd.DataFrame(index=daily_temp_avg.index)
df_daily_hdd_cdd['daily_avg_temp'] = daily_temp_avg

df_daily_hdd_cdd['hdd'] = (base_temperature_celsius - df_daily_hdd_cdd['daily_avg_temp']).apply(lambda x: max(0, x))
df_daily_hdd_cdd['cdd'] = (df_daily_hdd_cdd['daily_avg_temp'] - base_temperature_celsius).apply(lambda x: max(0, x))

# Map HDD and CDD back to your original hourly DataFrame
# Use .normalize() on the hourly index to match the daily index for mapping
df['hdd'] = df.index.normalize().map(df_daily_hdd_cdd['hdd'])
df['cdd'] = df.index.normalize().map(df_daily_hdd_cdd['cdd'])

# Handle potential NaNs such as if there were gaps or mismatches
# Use explicit assignment to avoid the FutureWarning
df['hdd'] = df['hdd'].fillna(0)
df['cdd'] = df['cdd'].fillna(0)

# Inspect the new features
print(df[['temperature', 'hdd', 'cdd']].head())
print(df[['temperature', 'hdd', 'cdd']].tail())

In [ ]:
# 1.2 Rolling Window Statistics
# Capture recent trends and volatility of the target variable, `PJME_MW` over specific time windows to
# give the model valuable context about the immediate past

# 24-hour (1-day) Rolling Mean of PJME_MW
# .rolling(window='24H') creates a rolling window of 24 hours.
# .mean() calculates the mean within that window.
# .bfill() is used here to fill NaN values at the beginning of the series, where there aren't enough
# previous data points to fill the window.
df['PJME_MW_rolling_24_hr_mean'] = df['PJME_MW'].rolling(window='24h', closed='left').mean().bfill()

# 168-hour (7-day) Rolling Mean of PJME_MW
df['PJME_MW_rolling_168_hr_mean'] = df['PJME_MW'].rolling(window='168h', closed='left').mean().bfill()

# 24-hour (1-day) Rolling Standard Deviation of PJME_MW
df['PJME_MW_rolling_24_hr_std'] = df['PJME_MW'].rolling(window='24h', closed='left').std().bfill()

# 168-hour (7-day) Rolling Standard Deviation of PJME_MW
df['PJME_MW_rolling_168_hr_std'] = df['PJME_MW'].rolling(window='168h', closed='left').std().bfill()

# Inspect the new features
print("\n--- After adding Rolling Mean Features ---")
print(df[['PJME_MW', 'PJME_MW_rolling_24_hr_mean', 'PJME_MW_rolling_168_hr_mean']].head(40)) # show more rows to see rolling effect
print(df[['PJME_MW', 'PJME_MW_rolling_24_hr_mean', 'PJME_MW_rolling_168_hr_mean']].tail())

In [ ]:
# 1.3 Interaction Features
# Created by combining two or more existing features to help the model capture relationships that are
# less apparent from individual features alone.

# Hour of day interacted with `is_weekend`
# Creates a unique value for each hour on a weekend vs on a weekday
# 1 is added to is_weekend (0 or 1) to distinguish weekend hours.
df['hour_of_day_x_is_weekend'] = df['hour_of_day'] * (df['is_weekend'] + 1)

# Temperature interacted with `is_holiday`
# Highlights temperature effects specifically on holidays
# 1 is added to is_holiday (0 or 1) to distinguish holiday temperatures.
df['temperature_x_is_holiday'] = df['temperature'] * (df['is_holiday'] + 1)

# Temperature interacted with `hour_of_day`
# Accounts for the varying effect of temperature on demand at different hours of the day
df['temperature_x_hour_of_day'] = df['temperature'] * (df['hour_of_day'] + 1)

# CDD interacted with `is_weekend`
# Captures the potentially different impact of cooling degree days (CDD) on weekends
# as compared to weekdays
df['cdd_x_is_weekend'] = df['cdd'] * df['is_weekend']

# Inspect the new features
print("\n--- After adding Interaction Features ---")
print(df[['hour_of_day', 'is_weekend', 'hour_of_day_x_is_weekend',
          'temperature', 'is_holiday', 'temperature_x_is_holiday']].head())
print(df[['hour_of_day', 'is_weekend', 'hour_of_day_x_is_weekend',
          'temperature', 'is_holiday', 'temperature_x_is_holiday']].tail())

In [ ]:
# Visualising selected advanced features

# Global plot aesthetics
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 8
plt.rcParams['figure.figsize'] = (15, 7) # Default figure size for horizontal plots

# HDD/CDD Visual: PJME_MW vs Heating Degree Days (HDD)
# Demonstrate the positive linear relationship between HDD and energy demand as heating needs increase
fig, ax = plt.subplots(figsize=(10, 7))

sns.scatterplot(
    data=df,
    x='hdd',
    y='PJME_MW',
    alpha=0.3,
    color='firebrick',
    ax=ax
)

ax.set_title('PJME_MW vs Heating Degree Days (HDD)')
ax.set_xlabel('HDD')
ax.set_ylabel('Energy (MW)')
plt.tight_layout()
plt.savefig('images/ph4_pjme_mw_vs_hdd.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

# HDD/CDD Visual: PJME_MW vs Cooling Degree Days (CDD)
# Show the positive linear relationship between CDD and energy demand as cooling needs increase
fig, ax = plt.subplots(figsize=(10, 7))

sns.scatterplot(
    data=df,
    x='cdd',
    y='PJME_MW',
    alpha=0.3,
    color='darkviolet',
    ax=ax
)

ax.set_title('PJME_MW vs Cooling Degree Days (CDD)')
ax.set_xlabel('CDD')
ax.set_ylabel('Energy (MW)')
plt.tight_layout()
plt.savefig('images/ph4_pjme_mw_vs_cdd.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

# Rolling Window Visual: PJME_MW vs its 24-Hour Rolling Mean
# Visualise the smoothing effect and how the rolling mean captures underlying trends
# A subset of data between June and August 2017 is plotted for clarity.
start_date_rolling = '2017-06-01'
end_date_rolling = '2017-08-31'

fig, ax = plt.subplots()

df.loc[start_date_rolling:end_date_rolling, 'PJME_MW'].plot(ax=ax,
                                            label='Actual PJME_MW',
                                            color='grey',
                                            alpha=0.6)
df.loc[start_date_rolling:end_date_rolling, 'PJME_MW_rolling_24_hr_mean'].plot(ax=ax,
                                                               label='24-Hour Rolling Mean',
                                                               color='maroon',
                                                               linestyle='--')
ax.set_title(f'PJME_MW vs 24-Hour Rolling Mean ({start_date_rolling} to {end_date_rolling})', fontsize=20)
ax.set_xlabel('Date')
ax.set_ylabel('Energy (MW)')
ax.legend(fontsize=12)
plt.tight_layout()
plt.savefig('images/ph4_pjme_mw_vs_24_hr_rolling_mean.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

# Rolling Window Visual: PJME_MW 24-Hour Rolling Standard Deviation
# Show periods of higher or lower volatility in energy demand as captured by the rolling standard deviation
fig, ax = plt.subplots()

df.loc[start_date_rolling:end_date_rolling, 'PJME_MW'].plot(ax=ax,
                                                            label='Actual PJME_MW',
                                                            color='saddlebrown',
                                                            alpha=0.6)
df.loc[start_date_rolling:end_date_rolling, 'PJME_MW_rolling_24_hr_mean'].plot(ax=ax,
                                                                               label='24-Hour Rolling Mean',
                                                                               color='navy',
                                                                               linestyle='--')
ax.set_title(f'PJME_MW vs 24-Hour Rolling Standard Deviation ({start_date_rolling} to {end_date_rolling})', fontsize=20)
ax.set_xlabel('Date')
ax.set_ylabel('Standard Deviation (MW)')
ax.legend(fontsize=12)
plt.tight_layout()
plt.savefig('images/ph4_pjme_mw_vs_24_hr_rolling_std_dev.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

# Interaction Feature: PJME_MW vs Is_Holiday
# Show the difference in energy consumption on holidays vs non-holidays
fig, ax = plt.subplots(figsize=(8, 6))

sns.boxplot(
    data=df,
    x='is_holiday',
    y='PJME_MW',
    hue='is_holiday',
    ax=ax,
    palette=['goldenrod', 'indigo'],
    legend=False
)

ax.set_title('PJME_MW by Holiday Status')
ax.set_xlabel('Is Holiday (0 = No, 1 = Yes)')
ax.set_ylabel('Energy (MW)')
plt.tight_layout()
plt.savefig('images/ph4_pjme_mw_vs_is_holiday.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

# Interaction Feature: PJME_MW vs CDD, colored by Is_Weekend
# Depict if the impact of cooling degree days on demand differs on weekends vs weekdays,
# highlighting the 'cdd_x_is_weekend' interaction
df['is_weekend_label'] = df['is_weekend'].map({0: 'No', 1: 'Yes'})
fig, ax = plt.subplots(figsize=(10, 7))

sns.scatterplot(
    data=df,
    x='cdd',
    y='PJME_MW',
    hue='is_weekend_label', # color points based on whether it's a weekend
    alpha=0.3,
    palette=['crimson', 'forestgreen'], # different palette for clarity
    ax=ax
)

ax.set_title('PJME_MW vs Cooling Degree Days (CDD) by Weekend Status')
ax.set_xlabel('CDD')
ax.set_ylabel('Energy (MW)')
plt.legend(title='Is Weekend?')
plt.tight_layout()
plt.savefig('images/ph4_pjme_mw_vs_cdd_by_is_weekend.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

In [ ]:
# Phase 5: Hyperparameter Optimisation using Optuna

# 1. Import Additional Libraries
# Install Optuna using CLI if it is not done already
# !pip install xgboost # for Colab
# !pip install lightgbm # for Colab
!pip install optuna
import xgboost
import lightgbm
import optuna
from sklearn.model_selection import TimeSeriesSplit

print("\nAdditional hyperparameter optimisation libraries imported successfully.")

In [ ]:
# 2. Data Splitting and Feature/Target Definition
# A similar data splitting approach to Phase 3 (Model Selection and Training) is adopted to prepare the refined
# dataset for model tuning and evaluation.

# Define the cutoff date
cutoff_date = '2017-01-01'

# Create the training and test sets
# .copy is used to avoid SettingWithCopyWarning
df_train = df.loc[df.index < cutoff_date].copy()
df_test = df.loc[df.index >= cutoff_date].copy()

# Define the features (X) and target (y)
# The target variable is 'PJME_MW'
target = 'PJME_MW'
exclude_features = ['is_weekend_label']

# List out all raw and newly engineering features excluding the target variable
# Check the list of feature columns: features = [col for col in df.columns if col != target and col not in exclude_features]

features = ['hour_of_day', 'day_of_week', 'day_of_year', 'week_of_year', 'month', 'quarter', 'year', 'is_weekend',
            'day_of_month', 'hour_sin', 'hour_cos', 'dayofyear_sin', 'dayofyear_cos', 'lag_1_hour', 'lag_24_hour',
            'is_holiday', 'temperature', 'lag_24_temp', 'rolling_72_temp_avg', 'temp_squared', 'hdd', 'cdd',
            'PJME_MW_rolling_24_hr_mean', 'PJME_MW_rolling_168_hr_mean',
            'PJME_MW_rolling_24_hr_std', 'PJME_MW_rolling_168_hr_std', 'hour_of_day_x_is_weekend',
            'temperature_x_is_holiday', 'temperature_x_hour_of_day', 'cdd_x_is_weekend']

# Create the final feature and target DataFrames for both training and testing
X_train = df_train[features]
y_train = df_train[target]
X_test = df_test[features]
y_test = df_test[target]

print(f"Training set has {X_train.shape[0]} rows and {X_train.shape[1]} columns.")
print(f"Test set has {X_test.shape[0]} rows and {X_test.shape[1]} columns.")

In [ ]:
# 3. Model Tuning

# --- XGBoost Model ---

# Define the objective function for Optuna to optimise hyperparameters
def xgb_objective(trial):
    """
    Define the objective function for Optuna to optimise hyperparameters for the XGBoost model.
    """

    # Check for NaN or infinite values in the data before starting the loop
    if X_train.isnull().values.any() or not np.isfinite(X_train).all().all():
        print("Error: X_train contains NaN or infinite values. Please clean your data.")
        return float('inf')
    if y_train.isnull().values.any() or not np.isfinite(y_train).all():
        print("Error: y_train contains NaN or infinite values. Please clean your data.")
        return float('inf')

    # Suggest hyperparameters to be optimised
    params = {
        'objective': 'reg:squarederror',
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'n_jobs': -1,
        'tree_method': 'hist',
        'eval_metric': 'rmse',
        'early_stopping_rounds': 50,
        'random_state': 42
    }

    # Train the model with suggested parameters
    model = xgb.XGBRegressor(**params)

    tscv = TimeSeriesSplit(n_splits=5)
    oof_predictions = np.zeros(len(X_train))

    # Track the number of folds that were successfully processed
    valid_folds = 0

    for fold, (train_index, val_index) in enumerate(tscv.split(X_train)):
      try:
        print(f"Trial {trial.number}, Fold {fold}: train_size={len(train_index)}, val_size={len(val_index)}")

        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        model.fit(X_train_fold, y_train_fold,
                  eval_set=[(X_val_fold, y_val_fold)],
                  verbose=False
                 )

        # Make predictions
        oof_predictions[val_index] = model.predict(X_val_fold)
        valid_folds =+ 1

        # Delete variables to manage memory
        del X_train_fold, X_val_fold, y_train_fold, y_val_fold

      except UnboundLocalError:
        # Continue if the error occurs due to the fold being invalid
        print(f"Trial {trial.number}, Fold {fold}: Skipping due to an invalid split.")
        continue

    # Return a high value to indicate a failed trial if no folds were created
    if valid_folds == 0:
      return float('inf')

    # Calculate the evaluation metric (RMSE)
    rmse = np.sqrt(mean_squared_error(y_train, oof_predictions))

    return rmse

In [ ]:
# Create an Optuna study and optimise the objective function

print("\nStarting hyperparameter optimisation with Optuna for XGBoost model...")
xgb_study = optuna.create_study(direction='minimize')
xgb_study.optimize(xgb_objective, n_trials=50, show_progress_bar=True)

In [ ]:
# View optimisation results

print("\n-- XGBoost Hyperparameter Optimisation Results --")
print("Best trial:")
print(f"  Value (RMSE): {xgb_study.best_trial.value:.2f}")
print("  Best Hyperparameters:")
for key, value in xgb_study.best_trial.params.items():
    print(f"  {key}:{value}")

In [ ]:
# Train the tuned model with best hyperparameters and evaluate
# Get the best hyperparameters from Optuna
best_xgb_params = xgb_study.best_trial.params

# Initialise and train the tuned XGBoost model with the best hyperparameters
# Train the model on the full training set, and use the test set for evaluation
tuned_xgb_model = xgb.XGBRegressor(**best_xgb_params)
tuned_xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

# Make predictions with the tuned model
tuned_xgb_y_pred = tuned_xgb_model.predict(X_test)

# Evaluate the tuned model's performance
tuned_xgb_rmse = np.sqrt(mean_squared_error(y_test, tuned_xgb_y_pred))
tuned_xgb_mae = mean_absolute_error(y_test, tuned_xgb_y_pred)
tuned_xgb_r2 = r2_score(y_test, tuned_xgb_y_pred)

print(f"XGBoost Tuned Model Performance:")
print(f"Root Mean Squared Error (RMSE): {tuned_xgb_rmse:.2f}")
print(f"Mean Absolute Error (MAE): {tuned_xgb_mae:.2f}")
print(f"R-squared (R^2): {tuned_xgb_r2:.5f}")

# Visualise Predictions vs Actuals
# Create a DataFrame for easy plotting
tuned_xgb_results = pd.DataFrame({'Actual': y_test, 'Predicted': tuned_xgb_y_pred}, index=y_test.index)

plt.figure(figsize=(15, 6))
plt.plot(tuned_xgb_results.index, tuned_xgb_results['Actual'], label='Actual Consumption', alpha=0.7)
plt.plot(tuned_xgb_results.index, tuned_xgb_results['Predicted'], label='Predicted Consumption', alpha=0.8)
plt.title('XGBoost Tuned Model: Actual vs Predicted Energy Consumption')
plt.xlabel('Datetime')
plt.ylabel('PJME_MW')
plt.legend()
plt.grid(True)
plt.savefig('images/ph5_xgboost_tuned_model.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

In [ ]:
# --- LightGBM Model ---

# Define the objective function for Optuna to optimise hyperparameters
def lgbm_objective(trial):
    """
    Define the objective function for Optuna to optimise hyperparameters for the LightGBM model.
    """
    # Suggest hyperparameters to be optimised
    params = {
        'objective': 'regression_l1',
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'metric': 'rmse',
        'early_stopping_rounds': 50,
        'random_state': 42
    }

    # Train the model with suggested parameters
    model = lgb.LGBMRegressor(**params)

    tscv = TimeSeriesSplit(n_splits=5)
    oof_predictions = np.zeros(len(X_train))

    # Track the number of folds that were successfully processed
    valid_folds = 0

    for fold, (train_index, val_index) in enumerate(tscv.split(X_train)):
      try:
        print(f"Trial {trial.number}, Fold {fold}: train_size={len(train_index)}, val_size={len(val_index)}")

        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        model.fit(X_train_fold, y_train_fold,
                  eval_set=[(X_val_fold, y_val_fold)],
                  callbacks=[early_stopping(stopping_rounds=50, verbose=False)]
                 )

        # Make predictions
        oof_predictions = model.predict(X_test)
        fold_count =+ 1

        # Delete variables to manage memory
        del X_train_fold, X_val_fold, y_train_fold, y_val_fold

      except UnboundLocalError:
        # Continue if the error occurs due to the fold being invalid
        print(f"Trial {trial.number}, Fold {fold}: Skipping due to an invalid split.")
        continue

    # Return a high value to indicate a failed trial if no folds were created
    if valid_folds == 0:
      return float('inf')

    rmse = np.sqrt(mean_squared_error(y_train, oof_predictions))

    return rmse

In [ ]:
# Create an Optuna study and optimise the objective function

print("\nStarting hyperparameter optimisation with Optuna for LightGBM model...")
lgbm_study = optuna.create_study(direction='minimize')
lgbm_study.optimize(lgbm_objective, n_trials=50, show_progress_bar=True)

print("\n-- LightGBM Hyperparameter Optimisation Results --")
print("Best trial:")
print(f"  Value (RMSE): {lgbm_study.best_trial.value:.2f}")
print("  Best Hyperparameters:")
for key, value in lgbm_study.best_trial.params.items():
    print(f"  {key}:{value}")

In [ ]:
# Train the tuned model with best hyperparameters and evaluate
# Get the best hyperparameters from Optuna
best_lgbm_params = lgbm_study.best_trial.params

# Initialise and train the tuned XGBoost model with the best hyperparameters
tuned_lgbm_model = xgb.XGBRegressor(**best_lgbm_params)
tuned_lgbm_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

# Make predictions with the tuned model
tuned_lgbm_y_pred = tuned_lgbm_model.predict(X_test)

# Evaluate the tuned model's performance
tuned_lgbm_rmse = np.sqrt(mean_squared_error(y_test, tuned_lgbm_y_pred))
tuned_lgbm_mae = mean_absolute_error(y_test, tuned_lgbm_y_pred)
tuned_lgbm_r2 = r2_score(y_test, tuned_lgbm_y_pred)

print(f"\nLightGBM Tuned Model Performance:")
print(f"Root Mean Squared Error (RMSE): {tuned_lgbm_rmse:.2f}")
print(f"Mean Absolute Error (MAE): {tuned_lgbm_mae:.2f}")
print(f"R-squared (R^2): {tuned_lgbm_r2:.5f}")

# Visualise Predictions vs Actuals
# Create a DataFrame for easy plotting
tuned_lgbm_results = pd.DataFrame({'Actual': y_test, 'Predicted': tuned_lgbm_y_pred}, index=y_test.index)

plt.figure(figsize=(15, 6))
plt.plot(tuned_lgbm_results.index, tuned_lgbm_results['Actual'], label='Actual Consumption', alpha=0.7)
plt.plot(tuned_lgbm_results.index, tuned_lgbm_results['Predicted'], label='Predicted Consumption', alpha=0.8)
plt.title('LightGBM Tuned Model: Actual vs Predicted Energy Consumption')
plt.xlabel('Datetime')
plt.ylabel('PJME_MW')
plt.legend()
plt.grid(True)
plt.savefig('images/ph5_lgbm_tuned_model.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook

In [ ]:
# Compare baseline and tuned models for both XGBoost and LightGBM

# Create a dictionary to store the performance metrics for each model
metrics = {
    'Baseline_XGBoost': {
        'RMSE': baseline_xgb_rmse,
        'MAE': baseline_xgb_mae,
        'R^2': baseline_xgb_r2
    },
    'Tuned_XGBoost': {
        'RMSE': tuned_xgb_rmse,
        'MAE': tuned_xgb_mae,
        'R^2': tuned_xgb_r2
    },
    'Baseline_LightGBM': {
        'RMSE': baseline_lgbm_rmse,
        'MAE': baseline_lgbm_mae,
        'R^2': baseline_lgbm_r2
    },
    'Tuned_LightGBM': {
        'RMSE': tuned_lgbm_rmse,
        'MAE': tuned_lgbm_mae,
        'R^2': tuned_lgbm_r2
    }
}

# Create a pandas DataFrame from the metrics dictionary for easy comparison
metrics_df = pd.DataFrame(metrics).transpose()
metrics_df = metrics_df.round(5)

print("---Model Performance Comparison Table ---")
print(metrics_df)

In [ ]:
# Create a pandas DataFrame for easy plotting
comparison_df = pd.DataFrame({
    'Actual': y_test,
    'Baseline_XGBoost': baseline_xgb_y_pred,
    'Tuned_XGBoost': tuned_xgb_y_pred,
    'Baseline_LightGBM': baseline_lgbm_y_pred,
    'Tuned_LightGBM': tuned_lgbm_y_pred
}, index=y_test.index)

plt.figure(figsize=(22, 10))

plt.plot(comparison_df.index, comparison_df['Actual'],
         label='Actual Consumption', color='black',
         alpha=0.9, linewidth=2.5)

plt.plot(comparison_df.index, comparison_df['Baseline_XGBoost'],
         label='XGBoost Baseline', color='royalblue',
         linestyle='--', alpha=0.7)
plt.plot(comparison_df.index, comparison_df['Tuned_XGBoost'],
         label='XGBoost Tuned', color='royalblue', alpha=0.8, linewidth=1.5)

plt.plot(comparison_df.index, comparison_df['Baseline_LightGBM'],
         label='LightGBM Baseline', color='darkorange',
         linestyle='--', alpha=0.7)
plt.plot(comparison_df.index, comparison_df['Tuned_LightGBM'],
         label='LightGBM Tuned', color='darkorange', alpha=0.8, linewidth=1.5)

plt.title('Comparison of Model Predictions vs Actual Energy Consumption', fontsize=18)
plt.xlabel('Datetime', fontsize=14)
plt.ylabel('PJME_MW', fontsize=14)
plt.legend(loc='upper left', fontsize=12)
plt.grid(True)
plt.tight_layout()

plt.savefig('images/ph5_baseline_tuned_model_comparison.png', dpi=300, bbox_inches='tight')
# plt.show() # uncomment this line to display visual while running the notebook